**Springboard DSCT: 8.3 - SQL Case Study - PART 2: SQLite**


**Initial Note:** Errors on the PHPMyAdmin server resulted in Q9 being answered in this notebook. Q8 included for context.


Setting up useful references:

In [1]:
from sqlalchemy import *
import pandas as pd
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


C:\Users\Kstou\AppData\Local\Temp/ipykernel_27736/4197245412.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


In [2]:
with engine.connect() as con:
    r = con.execute('SELECT * FROM Bookings')
    B_df = pd.DataFrame(r.fetchall())
    B_df.columns = r.keys()
print(B_df.head())

   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1


In [3]:
F_df = pd.read_sql_query('SELECT * FROM Facilities', engine)
print(F_df.head())

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  


In [4]:
M_df = pd.read_sql_query('SELECT * FROM Members', engine)
print(M_df.head())

   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  


In [5]:
print(B_df.info())
print(F_df.info())
print(M_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4043 entries, 0 to 4042
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   bookid     4043 non-null   int64 
 1   facid      4043 non-null   int64 
 2   memid      4043 non-null   int64 
 3   starttime  4043 non-null   object
 4   slots      4043 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 158.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   facid               9 non-null      int64  
 1   name                9 non-null      object 
 2   membercost          9 non-null      float64
 3   guestcost           9 non-null      float64
 4   initialoutlay       9 non-null      int64  
 5   monthlymaintenance  9 non-null      int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 560.0+ bytes
None


**Q8**: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than 30 Dollars. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

**Q9**: This time, produce the same result as in Q8, but using a subquery.

In [6]:
Q_9 = pd.read_sql_query("""SELECT firstname, surname, name AS Facility,
CASE WHEN memid = 0
THEN guestcost * slots
ELSE membercost * slots
END AS cost, starttime
FROM (SELECT * FROM Bookings AS b
JOIN Members AS m ON b.memid = m.memid
JOIN Facilities AS f ON b.facid = f.facid
WHERE starttime LIKE '2012-09-14%'
AND CASE WHEN m.memid = 0
THEN guestcost * slots
ELSE membercost * slots
END >30)
ORDER BY cost DESC;""", engine)
print(Q_9)

   firstname  surname        Facility   cost            starttime
0      GUEST    GUEST  Massage Room 2  320.0  2012-09-14 11:00:00
1      GUEST    GUEST  Massage Room 1  160.0  2012-09-14 09:00:00
2      GUEST    GUEST  Massage Room 1  160.0  2012-09-14 13:00:00
3      GUEST    GUEST  Massage Room 1  160.0  2012-09-14 16:00:00
4      GUEST    GUEST  Tennis Court 2  150.0  2012-09-14 17:00:00
5      GUEST    GUEST  Tennis Court 1   75.0  2012-09-14 16:00:00
6      GUEST    GUEST  Tennis Court 1   75.0  2012-09-14 19:00:00
7      GUEST    GUEST  Tennis Court 2   75.0  2012-09-14 14:00:00
8      GUEST    GUEST    Squash Court   70.0  2012-09-14 09:30:00
9     Jemima  Farrell  Massage Room 1   39.6  2012-09-14 14:00:00
10     GUEST    GUEST    Squash Court   35.0  2012-09-14 12:30:00
11     GUEST    GUEST    Squash Court   35.0  2012-09-14 15:00:00


**Q10:** Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [7]:
df_10 = pd.read_sql_query("""
SELECT name, CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots END AS cost 
FROM Bookings JOIN Members USING(memid) JOIN Facilities USING(facid);
""", engine)
print(df_10.describe())
df_10 = df_10.groupby('name').sum().sort_values('cost', ascending=False)
df_10 = df_10[df_10['cost']<1000]
df_10.rename(columns={'cost':'Total Revenue'}, inplace=True)
print(df_10)

              cost
count  4043.000000
mean     26.970245
std      56.086200
min       0.000000
25%       0.000000
50%       7.000000
75%      19.800000
max     640.000000
               Total Revenue
name                        
Pool Table             270.0
Snooker Table          240.0
Table Tennis           180.0


**Q11:** Produce a report of members and who recommended them in alphabetic surname,firstname order

In [8]:
df_11 = pd.read_sql_query("""
SELECT m1.memid, m1.firstname AS mem_firstname, m1.surname AS mem_surname, 
m2.memid AS rec_id, m2.firstname AS rec_firstname, m2.surname AS rec_surname
FROM Members AS m1
JOIN Members AS m2
ON m1.recommendedby = m2.memid
ORDER BY m1.firstname, m1.surname;
""", engine)
print(df_11)

    memid mem_firstname        mem_surname  rec_id rec_firstname rec_surname
0      21          Anna          Mackenzie       1        Darren       Smith
1      12          Anne              Baker       9        Ponder    Stibbons
2      10       Charles               Owen       1        Darren       Smith
3      11         David              Jones       4        Janice    Joplette
4      17         David             Pinker      13        Jemima     Farrell
5      26       Douglas              Jones      11         David       Jones
6      36         Erica            Crumpet       2         Tracy       Smith
7      15      Florence              Bader       9        Ponder    Stibbons
8       5        Gerald            Butters       1        Darren       Smith
9      27     Henrietta             Rumney      20       Matthew     Genting
10     29         Henry  Worthington-Smyth       2         Tracy       Smith
11     14          Jack              Smith       1        Darren       Smith

**Q12:** Find the facilities with their usage by member, but not guests

In [9]:
df_12 = pd.read_sql_query("""
SELECT b.facid, m.memid, m.firstname, m.surname 
FROM Bookings AS b 
JOIN Members AS m
USING(memid) 
WHERE b.memid != 0
GROUP BY facid, memid;
""", engine)
df_12 = df_12.sort_values('facid')
print(df_12)

     facid  memid  firstname     surname
0        0      2      Tracy       Smith
22       0     36      Erica     Crumpet
21       0     35       John        Hunt
20       0     28      David     Farrell
19       0     26    Douglas       Jones
..     ...    ...        ...         ...
176      8      2      Tracy       Smith
175      8      1     Darren       Smith
200      8     30  Millicent     Purview
187      8     13     Jemima     Farrell
201      8     33   Hyacinth  Tupperware

[202 rows x 4 columns]


**Q13:**

In [10]:
df_13 = pd.read_sql_query("""
SELECT facid, strftime('%m', starttime) AS month, memid, slots, firstname, surname 
FROM (SELECT b.facid, b.memid, b.starttime, b.slots, m.firstname, m.surname
FROM Bookings AS b 
JOIN Members AS m 
USING(memid))
WHERE memid != 0;
""", engine)
print(df_13)

      facid month  memid  slots firstname            surname
0         3    07      1      2    Darren              Smith
1         4    07      1      2    Darren              Smith
2         7    07      1      2    Darren              Smith
3         8    07      1      1    Darren              Smith
4         8    07      1      1    Darren              Smith
...     ...   ...    ...    ...       ...                ...
3155      8    09     29      2     Henry  Worthington-Smyth
3156      8    09     29      1     Henry  Worthington-Smyth
3157      8    09     21      1      Anna          Mackenzie
3158      8    09     16      1   Timothy              Baker
3159      8    09     29      1     Henry  Worthington-Smyth

[3160 rows x 6 columns]
